In [106]:
!pip install python-coinmarketcap
!pip install coinmarketcap
!pip install python-binance
!pip install yfinance


In [107]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Coincap

In [108]:
import requests
import time
import pandas as pd
json_ans = {"limit": 2000, "exchangeId":"binance"}
url = 'https://api.coincap.io/v2/markets'
r = requests.get(url, json_ans)


In [109]:
df = pd.DataFrame.from_records(r.json()['data'])

In [110]:
binance = df[df.exchangeId == 'binance']
binance = df[df.quoteSymbol == 'USDT']


In [111]:
binance.shape

(118, 12)

In [112]:
binance.head()

,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,tradesCount24Hr,updated
0,binance,1,BTT,bittorrent,USDT,tether,0.0119070000000000,0.0119086677427301,2774821628.1914975602536600,9.2399124937274944,1963585,1617618993336
1,binance,2,WIN,wink-tronbet,USDT,tether,0.0022970000000000,0.0022973217271396,2641441054.5284819802703819,8.7957668893800624,None,1617618996349
2,binance,3,BTC,bitcoin,USDT,tether,57390.4200000000000000,57398.4583350744902097,2463947641.1377644298490862,8.2047294002380584,1072391,1617619046373
3,binance,4,WRX,wazirx,USDT,tether,5.3761400000000000,5.3768930039809322,2416242729.2943634612514545,8.0458762305511493,1733114,1617618999864
4,binance,5,HOT,holo,USDT,tether,0.0279579000000000,0.0279618158969072,1136629609.5439387018791075,3.7848768451508005,1111143,1617618993532


In [113]:
binance[['baseSymbol', 'quoteSymbol', 'priceQuote', 'volumeUsd24Hr']].tail()

,baseSymbol,quoteSymbol,priceQuote,volumeUsd24Hr
267,IRIS,USDT,0.2204500000000000,4253082.1167522130482971
269,UMA,USDT,25.2390000000000000,4223710.7156707351252966
281,NXM,USDT,57.2470000000000000,3481646.5701304762847048
469,XZC,USDT,4.4460000000000000,746.5479358891767134
473,BULL,USDT,1370.2800000000000000,376.3658529458983785


In [114]:
binance[binance['baseSymbol'] == 'AUDIO']

,exchangeId,rank,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd,volumeUsd24Hr,percentExchangeVolume,tradesCount24Hr,updated
106,binance,107,AUDIO,audius,USDT,tether,2.8231200000000000,2.8235154176414024,32296428.1652639765111006,0.1075442713416823,81321,1617619009444


In [115]:
json_ans = {"limit": 2000}
url = 'https://api.coincap.io/v2/assets'
r = requests.get(url, json_ans)
coin_info = pd.DataFrame.from_records(r.json()['data'])



In [116]:
coin_info['marketCapUsd'] = coin_info['marketCapUsd'].astype(float)

In [117]:
coin_info[coin_info['symbol'] == 'MTL']

,id,rank,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr,priceUsd,changePercent24Hr,vwap24Hr,explorer
104,metal,105,MTL,Metal,65588845.1200000000000000,66588888.0000000000000000,3.947408e+08,86364464.8937622689646273,6.0184139576768096,-3.4108482695844821,6.2941821489649035,https://etherscan.io/token/0xF433089366899D83a...


In [118]:
# coin_info[coin_info['marketCapUsd'] < 10**6].head()

In [119]:
full_info = pd.merge(binance, coin_info, how='left', left_on=['baseSymbol'], right_on=['symbol'])


In [120]:
full_info.head()

,exchangeId,rank_x,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd_x,volumeUsd24Hr_x,percentExchangeVolume,tradesCount24Hr,updated,id,rank_y,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr_y,priceUsd_y,changePercent24Hr,vwap24Hr,explorer
0,binance,1,BTT,bittorrent,USDT,tether,0.0119070000000000,0.0119086677427301,2774821628.1914975602536600,9.2399124937274944,1963585,1617618993336,bittorrent,11,BTT,BitTorrent,989980422741.4906000000000000,None,1.200022e+10,4424224730.8914130703886240,0.0121216699707613,69.2512040870094721,0.0103623535872003,https://tronscan.org/#/token/1002000
1,binance,1,BTT,bittorrent,USDT,tether,0.0119070000000000,0.0119086677427301,2774821628.1914975602536600,9.2399124937274944,1963585,1617618993336,blocktrade,861,BTT,Blocktrade,55314915.0000000000000000,None,NaN,None,0.0037880119557663,None,None,https://etherscan.io/token/0xfa456cf55250a8390...
2,binance,2,WIN,wink-tronbet,USDT,tether,0.0022970000000000,0.0022973217271396,2641441054.5284819802703819,8.7957668893800624,None,1617618996349,wink-tronbet,83,WIN,WINk,313607571387.0000000000000000,None,7.206893e+08,2843415016.4040143099906378,0.0022980610209510,24.1801870377463315,None,https://tronscan.org/#/token20/TLa2f6VPqDgRE67...
3,binance,3,BTC,bitcoin,USDT,tether,57390.4200000000000000,57398.4583350744902097,2463947641.1377644298490862,8.2047294002380584,1072391,1617619046373,bitcoin,1,BTC,Bitcoin,18672456.0000000000000000,21000000.0000000000000000,1.071903e+12,15586003418.8762776861519502,57405.5449194301119865,-0.4194446595293871,57766.8352727535658133,https://blockchain.info/
4,binance,4,WRX,wazirx,USDT,tether,5.3761400000000000,5.3768930039809322,2416242729.2943634612514545,8.0458762305511493,1733114,1617618999864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
full_info

,exchangeId,rank_x,baseSymbol,baseId,quoteSymbol,quoteId,priceQuote,priceUsd_x,volumeUsd24Hr_x,percentExchangeVolume,tradesCount24Hr,updated,id,rank_y,symbol,name,supply,maxSupply,marketCapUsd,volumeUsd24Hr_y,priceUsd_y,changePercent24Hr,vwap24Hr,explorer
0,binance,1,BTT,bittorrent,USDT,tether,0.0119070000000000,0.0119086677427301,2774821628.1914975602536600,9.2399124937274944,1963585,1617618993336,bittorrent,11,BTT,BitTorrent,989980422741.4906000000000000,None,1.200022e+10,4424224730.8914130703886240,0.0121216699707613,69.2512040870094721,0.0103623535872003,https://tronscan.org/#/token/1002000
1,binance,1,BTT,bittorrent,USDT,tether,0.0119070000000000,0.0119086677427301,2774821628.1914975602536600,9.2399124937274944,1963585,1617618993336,blocktrade,861,BTT,Blocktrade,55314915.0000000000000000,None,NaN,None,0.0037880119557663,None,None,https://etherscan.io/token/0xfa456cf55250a8390...
2,binance,2,WIN,wink-tronbet,USDT,tether,0.0022970000000000,0.0022973217271396,2641441054.5284819802703819,8.7957668893800624,None,1617618996349,wink-tronbet,83,WIN,WINk,313607571387.0000000000000000,None,7.206893e+08,2843415016.4040143099906378,0.0022980610209510,24.1801870377463315,None,https://tronscan.org/#/token20/TLa2f6VPqDgRE67...
3,binance,3,BTC,bitcoin,USDT,tether,57390.4200000000000000,57398.4583350744902097,2463947641.1377644298490862,8.2047294002380584,1072391,1617619046373,bitcoin,1,BTC,Bitcoin,18672456.0000000000000000,21000000.0000000000000000,1.071903e+12,15586003418.8762776861519502,57405.5449194301119865,-0.4194446595293871,57766.8352727535658133,https://blockchain.info/
4,binance,4,WRX,wazirx,USDT,tether,5.3761400000000000,5.3768930039809322,2416242729.2943634612514545,8.0458762305511493,1733114,1617618999864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,binance,268,IRIS,irisnet,USDT,tether,0.2204500000000000,0.2204808771214285,4253082.1167522130482971,0.0141623901832712,21887,1617619004816,irisnet,145,IRIS,IRISnet,973341157.0890010000000000,None,2.147754e+08,12686033.9675573121495910,0.2206578564411673,1.5941088523464608,0.2176789414211729,https://www.irisplorer.io/#/home
121,binance,270,UMA,uma,USDT,tether,25.2390000000000000,25.2425350767418161,4223710.7156707351252966,0.0140645859954078,17701,1617619006862,uma,57,UMA,UMA,60008181.6973253800000000,101172570.0000000000000000,1.515385e+09,10520984.7395581471809871,25.2529804248016320,0.2791715893082100,25.3248097817212891,https://etherscan.io/token/0x04Fa0d235C4abf4Bc...
122,binance,282,NXM,nxm,USDT,tether,57.2470000000000000,57.2550182470873944,3481646.5701304762847048,0.0115935775169300,13368,1617619006082,nxm,111,NXM,NXM,6375672.8332371400000000,None,3.660876e+08,6302504.8395797325489138,57.4194420895444515,0.4212991386034107,59.1179181570286924,https://etherscan.io/token/0xd7c49cee7e9188cca...
123,binance,470,XZC,zcoin,USDT,tether,4.4460000000000000,4.4466227247986891,746.5479358891767134,0.0000024859391068,None,1617619001502,zcoin,550,XZC,Zcoin,11182443.3209889700000000,21400000.0000000000000000,4.971986e+07,746.4842432151861017,4.4462433556008726,-0.0022258885238620,None,http://explorer.zcoin.io


In [122]:
full_info = full_info[['baseSymbol', 'supply', 'maxSupply', 'marketCapUsd', 'priceQuote']]

In [123]:
full_info[['supply', 'maxSupply', 'marketCapUsd', 'priceQuote']] = full_info[['supply', 'maxSupply', 'marketCapUsd', 'priceQuote']].apply(pd.to_numeric)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [124]:
full_info = full_info.round(2)

In [125]:
full_info[full_info['baseSymbol'] == 'MLT']

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote


In [126]:
full_info[full_info['baseSymbol'] == 'MITH']

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
13,MITH,1.000000e+09,NaN,1.497483e+08,0.15


In [127]:
1.877655e+09	

1877655000.0

In [128]:
# full_info.dropna(subset = ['maxSupply'], inplace = True)
# full_info.dropna(subset = ['marketCapUsd'], inplace = True)

In [129]:
full_info[(full_info['marketCapUsd'] <= 20**8) & (full_info['priceQuote'] <= 0.1)]

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
0,BTT,9.899804e+11,NaN,1.200022e+10,0.01
2,WIN,3.136076e+11,NaN,7.206893e+08,0.00
5,HOT,1.691642e+11,NaN,4.709992e+09,0.03
6,HOT,0.000000e+00,NaN,0.000000e+00,0.03
12,DENT,9.369041e+10,NaN,1.973795e+09,0.02
23,FUN,1.029888e+10,NaN,5.521065e+08,0.05
24,NPXS,2.367449e+11,2.585266e+11,1.663195e+09,0.01
29,DOGE,1.290313e+11,NaN,7.498361e+09,0.06
31,KEY,4.468195e+09,NaN,1.280019e+08,0.03
32,KEY,0.000000e+00,NaN,0.000000e+00,0.03


In [130]:
filtered_df = full_info[(full_info['marketCapUsd'] <= 3*10**9)].sort_values(by=['marketCapUsd'])

In [131]:
filtered_df[filtered_df['baseSymbol'] == 'MITH']

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
13,MITH,1.000000e+09,NaN,1.497483e+08,0.15


In [132]:
filtered_df

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
73,SRM,0.000000e+00,1.000000e+10,0.000000e+00,6.29
6,HOT,0.000000e+00,NaN,0.000000e+00,0.03
7,TRX,0.000000e+00,NaN,0.000000e+00,0.14
112,FTT,0.000000e+00,NaN,0.000000e+00,40.38
109,NANO,0.000000e+00,1.332483e+08,0.000000e+00,5.30
...,...,...,...,...,...
65,HBAR,7.916866e+09,5.000000e+10,2.516260e+09,0.32
81,COMP,5.046750e+06,1.000000e+07,2.604997e+09,515.73
27,DASH,1.006039e+07,1.890000e+07,2.639346e+09,260.66
14,CHZ,5.586361e+09,8.888889e+09,2.676675e+09,0.48


In [133]:
filtered_df.to_csv("/content/gdrive/MyDrive/Trading/list_coin.csv", index = False)

In [134]:
full_info.dropna(subset = ['marketCapUsd'], inplace = True)
full_info.drop_duplicates(subset = ['baseSymbol'], inplace = True)


In [135]:
full_info

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
0,BTT,9.899804e+11,NaN,1.200022e+10,0.01
2,WIN,3.136076e+11,NaN,7.206893e+08,0.00
3,BTC,1.867246e+07,21000000.0,1.071903e+12,57390.42
5,HOT,1.691642e+11,NaN,4.709992e+09,0.03
7,TRX,0.000000e+00,NaN,0.000000e+00,0.14
...,...,...,...,...,...
119,CTXC,2.499387e+08,299792458.0,8.317122e+07,0.35
120,IRIS,9.733412e+08,NaN,2.147754e+08,0.22
121,UMA,6.000818e+07,101172570.0,1.515385e+09,25.24
122,NXM,6.375673e+06,NaN,3.660876e+08,57.25


In [136]:
full_info = full_info.sort_values(by=['marketCapUsd'])

In [137]:
full_info

,baseSymbol,supply,maxSupply,marketCapUsd,priceQuote
112,FTT,0.000000e+00,NaN,0.000000e+00,40.38
109,NANO,0.000000e+00,1.332483e+08,0.000000e+00,5.30
7,TRX,0.000000e+00,NaN,0.000000e+00,0.14
104,YFII,0.000000e+00,3.937500e+04,0.000000e+00,2787.44
73,SRM,0.000000e+00,1.000000e+10,0.000000e+00,6.29
...,...,...,...,...,...
20,ADA,3.194831e+10,4.500000e+10,3.738331e+10,1.17
19,DOT,9.809220e+08,NaN,4.269377e+10,43.46
10,BNB,1.545328e+08,1.705328e+08,5.390181e+10,348.60
8,ETH,1.153187e+08,NaN,2.349547e+11,2036.71


In [138]:
full_info.to_csv("/content/gdrive/MyDrive/Trading/list_coin.csv", index = False)